# Keopsys Amp Control
### Cell 1: Connect to instruments (focus box and ctrl+enter)

In [1]:
import visa
import time
import re
#from datetime import datetime as dt
#import numpy as np
rm = visa.ResourceManager()
#print(rm.list_resources())
#pm = rm.open_resource('USB0::0x1313::0x8078::P0019080::INSTR')
amp = rm.open_resource('ASRL4::INSTR',baud_rate = 19200,read_termination = '\r',write_termination = '\r')
#laser = rm.open_resource('GPIB0::5::INSTR') #Tunics set to GPIB channel 5
#pmInfo = pm.query('*IDN?')
#pmHeadInfo = pm.query('SYST:SENS:IDN?')
#laserInfo = laser.query('*IDN?')
ampThreshold = re.search('[\-,0-9]+', amp.query('ASI?'))
ampThreshold = float(ampThreshold[0])
ampMode = re.search('[\-,0-9]+', amp.query('ASS?'))
ampMode = int(ampMode[0])
ampInput = re.search('[\-,0-9]+', amp.query('PUE?'))
ampInput = float(ampInput[0])/100
ampOutput = re.search('[\-,0-9]+', amp.query('PUS?'))
ampOutput = float(ampOutput[0])/100
if ampMode == 0:
    ampState = 'Off'
else:
    ampState = 'On'
print('Pump is %s'%ampState)
print('Power In  = %.3f mW'%10**(ampInput/10))
print('Power Out = %.3f mW'%10**(ampOutput/10))

SAFETY = 4000

Pump is On
Power In  = 0.418 mW
Power Out = 124.165 mW


### Cell 2: Toggle On/Off (focus box and ctrl+enter)

In [8]:
ampMode = re.search('[\-,0-9]+', amp.query('ASS?'))
ampMode = int(ampMode[0])
ampInput = re.search('[\-,0-9]+', amp.query('PUE?'))
ampInput = float(ampInput[0])/100
ampOutput = re.search('[\-,0-9]+', amp.query('PUS?'))
ampOutput = float(ampOutput[0])/100

if ampMode != 0:
    amp.query('ASS=0')
    print('Turning Off...')
    time.sleep(2)
    print('Pump Off.')
elif ampMode == 0:
    if ampInput < ampThreshold:
        print('Seed power too low. Pump is off.')
    else:
        amp.query('ASS=1')
        print('Warming up...')
        time.sleep(10)
        print('Pump On.')
        
ampMode = re.search('[\-,0-9]+', amp.query('ASS?'))
ampMode = int(ampMode[0])
ampOutput = re.search('[\-,0-9]+', amp.query('PUS?'))
ampOutput = float(ampOutput[0])/100
print('Power Out = %.3f mW'%10**(ampOutput/10))

Warming up...
Pump On.
Power Out = 74.131 mW


### Cell 3: Change Current (focus box and ctrl+enter)

In [2]:
CURRENT = 1500# mA. ~~~ Max is UNKNOWN. Increase SLOWLY.

if CURRENT > SAFETY:
    print('Too high. Choose a lower current.')
ampCurrent = re.search('(IC2=)([\-,0-9]+)', amp.query('IC2?'))
ampCurrent = int(ampCurrent.group(2))
print('Old current = %i mA'%ampCurrent)
amp.query('IC2=%i'%CURRENT)
ampCurrent = re.search('(IC2=)([\-,0-9]+)', amp.query('IC2?'))
ampCurrent = int(ampCurrent.group(2))
print('New current = %i mA'%ampCurrent)
if ampMode != 0:
    time.sleep(2)
    ampOutput = re.search('[\-,0-9]+', amp.query('PUS?'))
    ampOutput = float(ampOutput[0])/100
    print('Power Out = %.3f mW'%10**(ampOutput/10))
    print('Done.')
else:
    print('Done.')

Old current = 1200 mA
New current = 1500 mA
Power Out = 195.884 mW
Done.


### Free Resources And Disconnect

In [3]:
#laser.close()
amp.close()
#pm.close()

('ASRL3::INSTR', 'ASRL4::INSTR')
